In [1]:
import random
import os
import re
import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.expand_frame_repr', False)  # DataFrameを改行せずに表示
pd.set_option('display.max_columns', None)  # すべての列を表示

In [2]:
rootpath = "/Prove/Yoshihisa/HEIF_ghost/QPD/"

single_train_csv_path1 = os.path.join(rootpath, 'QPD11_HEIF_images_single_csv', 'TRAINING')
single_train_csv_path2 = os.path.join(rootpath, 'QPD11_HEIF_images_second_sameQP_csv', 'TRAINING')

second_train_csv_path1 = os.path.join(rootpath, 'QPD11_HEIF_images_second_csv', 'TRAINING')
second_train_csv_path2 = os.path.join(rootpath, 'QPD11_HEIF_images_triple_csv', 'TRAINING')

single_test_csv_path1 = os.path.join(rootpath, 'QPD11_HEIF_images_single_csv', 'TEST')
single_test_csv_path2 = os.path.join(rootpath, 'QPD11_HEIF_images_second_sameQP_csv', 'TEST')

second_test_csv_path1 = os.path.join(rootpath, 'QPD11_HEIF_images_second_csv', 'TEST')
second_test_csv_path2 = os.path.join(rootpath, 'QPD11_HEIF_images_triple_csv', 'TEST')


single_train_csv_path1_list = [os.path.join(single_train_csv_path1, file) for file in sorted(os.listdir(single_train_csv_path1))]
single_train_csv_path2_list = [os.path.join(single_train_csv_path2, file) for file in sorted(os.listdir(single_train_csv_path2))]

second_train_csv_path1_list = [os.path.join(second_train_csv_path1, file) for file in sorted(os.listdir(second_train_csv_path1))]
second_train_csv_path2_list = [os.path.join(second_train_csv_path2, file) for file in sorted(os.listdir(second_train_csv_path2))]

single_test_csv_path1_list = [os.path.join(single_test_csv_path1, file) for file in sorted(os.listdir(single_test_csv_path1))]
single_test_csv_path2_list = [os.path.join(single_test_csv_path2, file) for file in sorted(os.listdir(single_test_csv_path2))]

second_test_csv_path1_list = [os.path.join(second_test_csv_path1, file) for file in sorted(os.listdir(second_test_csv_path1))]
second_test_csv_path2_list = [os.path.join(second_test_csv_path2, file) for file in sorted(os.listdir(second_test_csv_path2))]


print("single_train_csv_path1_list: ", len(single_train_csv_path1_list))
print("single_train_csv_path2_list: ", len(single_train_csv_path2_list))

print("second_train_csv_path1_list: ", len(second_train_csv_path1_list))
print("second_train_csv_path2_list: ", len(second_train_csv_path2_list))

print("single_test_csv_path1_list: ", len(single_test_csv_path1_list))
print("single_test_csv_path2_list: ", len(single_test_csv_path2_list))

print("second_test_csv_path1_list: ", len(second_test_csv_path1_list))
print("second_test_csv_path2_list: ", len(second_test_csv_path2_list))


single_train_csv_path1_list:  270
single_train_csv_path2_list:  270
second_train_csv_path1_list:  270
second_train_csv_path2_list:  270
single_test_csv_path1_list:  30
single_test_csv_path2_list:  30
second_test_csv_path1_list:  30
second_test_csv_path2_list:  30


In [3]:
single_train_csv = list(zip(single_train_csv_path1_list, single_train_csv_path2_list))
second_train_csv = list(zip(second_train_csv_path1_list, second_train_csv_path2_list))
second_train_csv = random.sample(second_train_csv, len(single_train_csv))

single_test_csv = list(zip(single_test_csv_path1_list, single_test_csv_path2_list))
second_test_csv = list(zip(second_test_csv_path1_list, second_test_csv_path2_list))
second_test_csv = random.sample(second_test_csv, len(single_test_csv))

train_csv_list = single_train_csv + second_train_csv
test_csv_list = single_test_csv + second_test_csv

print("train_csv_list: ", len(train_csv_list))
print("test_csv_list: ", len(test_csv_list))

train_csv_list:  540
test_csv_list:  60


In [4]:
# new_df = pd.DataFrame(columns=["QP", "CU_64", "CU_32", "CU_16", "CU_8", "PU_64", "PU_32", "PU_16", "PU_8", "PU_4", "LUM_A", "LUM_B", "LUM_C", "CRM_34", "LABEL"])
# train_df1 = pd.DataFrame(columns=["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4", "LABEL"])
train_df1 = pd.DataFrame(columns=["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"])
train_df2 = pd.DataFrame(columns=["LABEL"])
scaler = MinMaxScaler()

for path1, path2 in train_csv_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path2) else 0
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path2)
    
#     cu1_64 = df1.loc[0, "depth_counts"]
#     cu1_32 = df1.loc[1, "depth_counts"]
#     cu1_16 = df1.loc[2, "depth_counts"]
#     cu1_8 = df1.loc[3, "depth_counts"]
    
#     cu2_64 = df2.loc[0, "depth_counts"]
#     cu2_32 = df2.loc[1, "depth_counts"]
#     cu2_16 = df2.loc[2, "depth_counts"]
#     cu2_8 = df2.loc[3, "depth_counts"]

    pu1_64 = df1.loc[0, "pu_counts"]
    pu1_32 = df1.loc[1, "pu_counts"]
    pu1_16 = df1.loc[2, "pu_counts"]
    pu1_8 = df1.loc[3, "pu_counts"]
    pu1_4 = df1.loc[4, "pu_counts"]
    
    pu2_64 = df2.loc[0, "pu_counts"]
    pu2_32 = df2.loc[1, "pu_counts"]
    pu2_16 = df2.loc[2, "pu_counts"]
    pu2_8 = df2.loc[3, "pu_counts"]
    pu2_4 = df2.loc[4, "pu_counts"]
    
    train_df1 = pd.concat([train_df1, pd.DataFrame({
                                          "PU1_64": [pu1_64],
                                          "PU1_32": [pu1_32],
                                          "PU1_16": [pu1_16],
                                          "PU1_8": [pu1_8],
                                          "PU1_4": [pu1_4],
                                          
                                          "PU2_64": [pu2_64],
                                          "PU2_32": [pu2_32],
                                          "PU2_16": [pu2_16],
                                          "PU2_8": [pu2_8],
                                          "PU2_4": [pu2_4],

                                          # "LABEL": [label]
    })], 
                   ignore_index=True)
    
    train_df2 = pd.concat([train_df2, pd.DataFrame({

                                          "LABEL": [label]})], 
                   ignore_index=True)
    
print(len(train_df1))

540


In [5]:
# test_df1 = pd.DataFrame(columns=["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4", "LABEL"])
test_df1 = pd.DataFrame(columns=["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"])
test_df2 = pd.DataFrame(columns=["LABEL"])
scaler = MinMaxScaler()

for path1, path2 in test_csv_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path2) else 0

    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path2)

#     cu1_64 = df1.loc[0, "depth_counts"]
#     cu1_32 = df1.loc[1, "depth_counts"]
#     cu1_16 = df1.loc[2, "depth_counts"]
#     cu1_8 = df1.loc[3, "depth_counts"]
    
#     cu2_64 = df2.loc[0, "depth_counts"]
#     cu2_32 = df2.loc[1, "depth_counts"]
#     cu2_16 = df2.loc[2, "depth_counts"]
#     cu2_8 = df2.loc[3, "depth_counts"]

    pu1_64 = df1.loc[0, "pu_counts"]
    pu1_32 = df1.loc[1, "pu_counts"]
    pu1_16 = df1.loc[2, "pu_counts"]
    pu1_8 = df1.loc[3, "pu_counts"]
    pu1_4 = df1.loc[4, "pu_counts"]
    
    pu2_64 = df2.loc[0, "pu_counts"]
    pu2_32 = df2.loc[1, "pu_counts"]
    pu2_16 = df2.loc[2, "pu_counts"]
    pu2_8 = df2.loc[3, "pu_counts"]
    pu2_4 = df2.loc[4, "pu_counts"]
    
    test_df1 = pd.concat([test_df1, pd.DataFrame({
                                          "PU1_64": [pu1_64],
                                          "PU1_32": [pu1_32],
                                          "PU1_16": [pu1_16],
                                          "PU1_8": [pu1_8],
                                          "PU1_4": [pu1_4],
                                          
                                          "PU2_64": [pu2_64],
                                          "PU2_32": [pu2_32],
                                          "PU2_16": [pu2_16],
                                          "PU2_8": [pu2_8],
                                          "PU2_4": [pu2_4],

                                          # "LABEL": [label]
    })], 
                   ignore_index=True)
    
    
    test_df2 = pd.concat([test_df2, pd.DataFrame({

                                          "LABEL": [label]})], 
                   ignore_index=True)

print(len(test_df1))

60


In [6]:
combined_df = pd.concat([train_df1, test_df1], ignore_index=True)
# print(len(combined_df))

# スケーラーを使って結合したデータをスケーリング
combined_scaled_data = scaler.fit_transform(combined_df)

# トレーニングデータとテストデータに再分割
X_train = combined_scaled_data[:len(train_df1)]
X_test = combined_scaled_data[len(train_df1):]
# print(len(X_train))
# print(len(X_test))

# ラベルの準備
Y_train = train_df2['LABEL'].astype(int)
Y_test = test_df2['LABEL'].astype(int)


In [7]:
print(X_train[0:1])
print(X_train[1350:1351])

[[0.         0.1715847  0.46609125 0.63167104 0.42520103 0.
  0.17030568 0.46994536 0.62080975 0.41602625]]
[]


In [8]:
# print(test_df1[0:5])
# print(test_df1[150:155])

# X_test = scaler.fit_transform(test_df1)  
# Y_test = test_df2['LABEL'].astype(int)

print(X_test[0:5])
print(X_test[150:155])

[[0.         0.1442623  0.72133169 0.6746865  0.28815057 0.
  0.1419214  0.73649059 0.66216805 0.27571022]
 [0.         0.37377049 0.89889026 0.41586468 0.04630202 0.
  0.37991266 0.90892532 0.38441445 0.04170624]
 [0.         0.53005464 0.44636252 0.29760863 0.17522312 0.
  0.53275109 0.44080146 0.28892759 0.17140143]
 [0.         0.78579235 0.26942047 0.14158647 0.04038173 0.
  0.79148472 0.26168792 0.13147584 0.03954753]
 [0.         0.65245902 0.25462392 0.28331875 0.13545993 0.
  0.64519651 0.26958106 0.27862429 0.13263103]]
[]


In [9]:
# Cの範囲を指定
C_values = {'C': [0.01, 0.1, 1, 10, 100, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]}

# SVMモデルのインスタンスを作成
svm_model = SVC(kernel='rbf')

# グリッドサーチのインスタンスを作成
grid_search = GridSearchCV(svm_model, C_values, cv=9, scoring='accuracy')

# グリッドサーチを実行
grid_search.fit(X_train, Y_train)

# 結果のデータフレームを作成
results = pd.DataFrame(grid_search.cv_results_)
# print(results)

# 新しい列名のマッピングを作成
new_column_names = {
    'param_C': 'C',
    'split0_test_score': 'k=1',
    'split1_test_score': 'k=2',
    'split2_test_score': 'k=3',
    'split3_test_score': 'k=4',
    'split4_test_score': 'k=5',
    'split5_test_score': 'k=6',
    'split6_test_score': 'k=7',
    'split7_test_score': 'k=8',
    'split8_test_score': 'k=9',
    'mean_test_score': 'Mean_Val_Score'
}

# 列名を変更
results = results.rename(columns=new_column_names)

# 変更後の表を表示
print(results[['C', 'k=1', 'k=2', 'k=3', 'k=4', 'k=5', 'k=6', 'k=7', 'k=8', 'k=9', 'Mean_Val_Score']])
    
# # 最適なハイパーパラメータを表示
print("Best Parameters: ", grid_search.best_params_)

# # 最適なモデルを取得
best_svm_model = grid_search.best_estimator_

# # テストデータで評価
accuracy = best_svm_model.score(X_test, Y_test)
print("Accuracy on Test Set: {:.4f}".format(accuracy))

       C       k=1       k=2       k=3       k=4       k=5       k=6       k=7       k=8       k=9  Mean_Val_Score
0   0.01  0.783333  0.616667  0.550000  0.733333  0.683333  0.666667  0.600000  0.600000  0.633333        0.651852
1    0.1  0.766667  0.616667  0.566667  0.716667  0.733333  0.683333  0.566667  0.600000  0.666667        0.657407
2      1  0.783333  0.683333  0.600000  0.683333  0.733333  0.633333  0.683333  0.583333  0.650000        0.670370
3     10  0.766667  0.766667  0.633333  0.666667  0.733333  0.616667  0.716667  0.600000  0.733333        0.692593
4    100  0.833333  0.900000  0.716667  0.800000  0.883333  0.733333  0.783333  0.783333  0.833333        0.807407
5   1000  0.866667  0.916667  0.766667  0.916667  0.883333  0.816667  0.900000  0.850000  0.866667        0.864815
6   1500  0.866667  0.933333  0.783333  0.933333  0.900000  0.850000  0.933333  0.833333  0.883333        0.879630
7   2000  0.900000  0.900000  0.783333  0.933333  0.916667  0.850000  0.933333  

In [10]:
# PU1 + PU2

k = 9

# SVMモデルを初期化（RBFカーネルを使用）
svm_model = SVC(kernel='rbf', C=4500, gamma='scale')  # Cとgammaはハイパーパラメータで調整可能


# K-fold cross validation
cv_scores = cross_val_score(svm_model, X_train, Y_train, cv=k)
average_accuracy = np.round(cv_scores.mean(), 4)
print(f'Average validation performance of {k}-fold: {average_accuracy}')

svm_model.fit(X_train, Y_train)
Y_pred = svm_model.predict(X_test)

report = classification_report(Y_test, Y_pred)

print(f'Summary:\n{report}')

svm_model._gamma

Average validation performance of 9-fold: 0.8944
Summary:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        30
           1       0.90      0.90      0.90        30

    accuracy                           0.90        60
   macro avg       0.90      0.90      0.90        60
weighted avg       0.90      0.90      0.90        60



1.2563589942375788